# BCI Training & Fine-Tuning Pipeline Test - Google Colab Version

Este notebook testa o pipeline completo no Google Colab:
1. **Setup**: Instalar dependências e clonar repositório
2. **Treinamento Principal**: Subjects 1-79 do dataset PhysioNet
3. **Fine-Tuning**: Dados específicos da pasta Davi
4. **Validação**: Métricas e comparação de performance

## ⚠️ IMPORTANTE: Este notebook foi adaptado para Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Setup Inicial no Google Colab

Vamos instalar as dependências e configurar o ambiente adequadamente.

In [ ]:
# === INSTALAÇÃO DE DEPENDÊNCIAS ===
print("🔧 Instalando dependências necessárias...")

# Instalar braindecode e dependências científicas
!pip install braindecode mne torch torchvision torchaudio
!pip install scikit-learn pandas numpy matplotlib seaborn
!pip install scipy jupyter

print("✅ Dependências instaladas!")

# Verificar se está no Colab
try:
    import google.colab
    IN_COLAB = True
    print("📱 Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Executando localmente")

🔧 Instalando dependências necessárias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.3 MB/s eta 0:00:00
   ━━

In [ ]:
# === CLONE DO REPOSITÓRIO (OPCIONAL) ===
import os
from pathlib import Path

if IN_COLAB:
    print("🔄 Configurando para Google Colab...")

    # Se você quiser clonar seu repositório:
    # !git clone https://github.com/seu-usuario/projetoBCI.git
    # os.chdir('/content/projetoBCI')

    # Por enquanto, vamos trabalhar no diretório padrão do Colab
    project_root = Path('/content')

    # Criar estrutura de diretórios
    (Path("/content/drive/MyDrive/Colab Notebooks/eeg_data")).mkdir(exist_ok=True)
    (project_root / "models").mkdir(exist_ok=True)
    (project_root / "results").mkdir(exist_ok=True)

else:
    # Executando localmente
    project_root = Path(os.getcwd()).parent

print(f"📂 Diretório de trabalho: {project_root}")
print(f"📁 Estrutura criada!")

🔄 Configurando para Google Colab...
📂 Diretório de trabalho: /content
📁 Estrutura criada!


## 2. Implementação das Classes Base (Inline)

Já que não temos o repositório clonado, vamos implementar as classes principais inline.

In [ ]:
# === IMPLEMENTAÇÃO INLINE DAS CLASSES BASE ===
import torch
import torch.nn as nn
import numpy as np
from scipy import signal
from typing import Optional, List, Tuple, Dict, Union
import pandas as pd
from datetime import datetime
import json
import logging
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports básicos carregados!")

✅ Imports básicos carregados!


In [ ]:
# === UNIVERSAL EEG NORMALIZER ===
class UniversalEEGNormalizer:
    """Normalizador universal para dados EEG"""

    def __init__(self, method: str = 'zscore', mode: str = 'training'):
        self.method = method
        self.mode = mode
        self.global_stats = {}
        self.is_fitted = False

    def _ensure_3d(self, data: np.ndarray) -> np.ndarray:
        """Garantir que dados estejam em 3D (n_samples, n_channels, n_timepoints)"""
        if len(data.shape) == 2:
            n_samples, n_features = data.shape
            if n_features % 16 == 0:  # Assumir 16 canais
                n_channels = 16
                n_timepoints = n_features // n_channels
                data = data.reshape(n_samples, n_channels, n_timepoints)
            else:
                data = data[:, np.newaxis, :]
        elif len(data.shape) == 1:
            data = data[np.newaxis, np.newaxis, :]
        return data

    def fit(self, data: np.ndarray):
        """Ajustar normalizador aos dados"""
        data_3d = self._ensure_3d(data)

        if self.method == 'zscore':
            self.global_stats['mean'] = np.mean(data_3d, axis=(0, 2), keepdims=True)
            self.global_stats['std'] = np.std(data_3d, axis=(0, 2), keepdims=True)
            self.global_stats['std'] = np.where(self.global_stats['std'] == 0, 1.0, self.global_stats['std'])

        self.is_fitted = True
        return self

    def transform(self, data: np.ndarray) -> np.ndarray:
        """Transformar dados"""
        if not self.is_fitted:
            raise ValueError("Normalizador deve ser ajustado antes da transformação")

        original_shape = data.shape
        data_3d = self._ensure_3d(data)

        if self.method == 'zscore':
            normalized = (data_3d - self.global_stats['mean']) / self.global_stats['std']

        # Restaurar forma original
        if len(original_shape) == 2:
            return normalized.reshape(original_shape[0], -1)
        elif len(original_shape) == 1:
            return normalized.flatten()
        return normalized

    def fit_transform(self, data: np.ndarray) -> np.ndarray:
        """Ajustar e transformar em um passo"""
        return self.fit(data).transform(data)

print("✅ UniversalEEGNormalizer implementado!")

✅ UniversalEEGNormalizer implementado!


# Normalização Melhorada para EEG

Implementação de um normalizador mais robusto com:
- Múltiplas estratégias de normalização
- Validação de qualidade
- Tratamento de outliers
- Normalização por canal ou por trial

In [ ]:
# === IMPROVED EEG NORMALIZER ===
import numpy as np
from typing import Dict, Optional, Tuple
from sklearn.preprocessing import RobustScaler


class ImprovedEEGNormalizer:
    """Normalizador EEG avançado com múltiplas estratégias e validação"""

    def __init__(self, method: str = 'robust_zscore', scope: str = 'channel',
                 outlier_threshold: float = 3.0):
        """
        Args:
            method: 'robust_zscore', 'minmax', ou 'raw_zscore'
            scope: 'channel', 'trial', ou 'global'
            outlier_threshold: número de desvios para considerar outlier
        """
        self.method = method
        self.scope = scope
        self.outlier_threshold = outlier_threshold
        self.stats: Dict = {}
        self.is_fitted = False

    def _handle_outliers(self, X: np.ndarray) -> np.ndarray:
        """Detecta e trata outliers usando IQR ou desvio padrão"""
        if self.method == 'robust_zscore':
            Q1 = np.percentile(X, 25, axis=(0, 2), keepdims=True)
            Q3 = np.percentile(X, 75, axis=(0, 2), keepdims=True)
            IQR = Q3 - Q1
            lower = Q1 - self.outlier_threshold * IQR
            upper = Q3 + self.outlier_threshold * IQR
        else:
            mean = np.mean(X, axis=(0, 2), keepdims=True)
            std = np.std(X, axis=(0, 2), keepdims=True)
            lower = mean - self.outlier_threshold * std
            upper = mean + self.outlier_threshold * std

        # Clip valores extremos
        return np.clip(X, lower, upper)

    def fit(self, X: np.ndarray) -> 'ImprovedEEGNormalizer':
        """Ajusta o normalizador aos dados

        Args:
            X: Array (trials, channels, time) ou (trials, features)
        """
        # Garantir formato 3D
        if len(X.shape) == 2:
            if X.shape[1] % 16 == 0:  # Assumir 16 canais
                n_channels = 16
                X = X.reshape(X.shape[0], n_channels, -1)
            else:
                X = X[:, np.newaxis, :]

        # Tratar outliers
        X = self._handle_outliers(X)

        if self.scope == 'channel':
            if self.method == 'robust_zscore':
                self.stats['median'] = np.median(X, axis=(0, 2), keepdims=True)
                q75, q25 = np.percentile(X, [75, 25], axis=(0, 2))
                self.stats['iqr'] = (q75 - q25)[None, :, None] + 1e-8

            elif self.method == 'minmax':
                self.stats['min'] = X.min(axis=(0, 2), keepdims=True)
                self.stats['max'] = X.max(axis=(0, 2), keepdims=True)

            else:  # raw_zscore
                self.stats['mean'] = np.mean(X, axis=(0, 2), keepdims=True)
                self.stats['std'] = np.std(X, axis=(0, 2), keepdims=True) + 1e-8

        elif self.scope == 'trial':
            if self.method == 'robust_zscore':
                self.stats['median'] = np.median(X, axis=2, keepdims=True)
                q75, q25 = np.percentile(X, [75, 25], axis=2)
                self.stats['iqr'] = (q75 - q25)[:, :, None] + 1e-8

            elif self.method == 'minmax':
                self.stats['min'] = X.min(axis=2, keepdims=True)
                self.stats['max'] = X.max(axis=2, keepdims=True)

            else:  # raw_zscore
                self.stats['mean'] = np.mean(X, axis=2, keepdims=True)
                self.stats['std'] = np.std(X, axis=2, keepdims=True) + 1e-8

        self.is_fitted = True
        return self

    def transform(self, X: np.ndarray) -> np.ndarray:
        """Transforma os dados usando as estatísticas calculadas"""
        if not self.is_fitted:
            raise ValueError("Normalize.fit() deve ser chamado antes de transform()")

        # Garantir formato 3D
        original_shape = X.shape
        if len(X.shape) == 2:
            if X.shape[1] % 16 == 0:
                n_channels = 16
                X = X.reshape(X.shape[0], n_channels, -1)
            else:
                X = X[:, np.newaxis, :]

        # Transformação
        if self.method == 'robust_zscore':
            X_norm = (X - self.stats['median']) / self.stats['iqr']
        elif self.method == 'minmax':
            X_norm = (X - self.stats['min']) / (self.stats['max'] - self.stats['min'] + 1e-8)
        else:  # raw_zscore
            X_norm = (X - self.stats['mean']) / self.stats['std']

        # Restaurar forma original se necessário
        if len(original_shape) == 2:
            X_norm = X_norm.reshape(original_shape)

        return X_norm

    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        """Ajusta aos dados e transforma em um único passo"""
        return self.fit(X).transform(X)

    def get_stats(self) -> Dict:
        """Retorna estatísticas de normalização"""
        return self.stats.copy()


def validate_normalization(X: np.ndarray, normalized_X: np.ndarray) -> Tuple[bool, Dict]:
    """Valida a qualidade da normalização

    Args:
        X: Dados originais
        normalized_X: Dados normalizados

    Returns:
        (is_valid, stats): Booleano indicando se passou nos checks e estatísticas
    """
    stats = {}

    # 1. Verificar média e desvio
    stats['mean'] = float(np.mean(normalized_X))
    stats['std'] = float(np.std(normalized_X))

    # 2. Calcular percentis
    stats['percentiles'] = {
        '1%': float(np.percentile(normalized_X, 1)),
        '99%': float(np.percentile(normalized_X, 99))
    }

    # 3. Verificar preservação de ordem relativa
    original_order = np.argsort(np.mean(X, axis=(1,2)))
    normalized_order = np.argsort(np.mean(normalized_X, axis=(1,2)))
    stats['order_correlation'] = float(np.corrcoef(original_order, normalized_order)[0,1])

    # 4. Verificar outliers
    z_scores = np.abs((normalized_X - np.mean(normalized_X)) / np.std(normalized_X))
    stats['outliers_ratio'] = float(np.mean(z_scores > 3))

    # Critérios de validação
    is_valid = (
        abs(stats['mean']) < 0.1 and           # Média próxima de zero
        abs(stats['std'] - 1) < 0.5 and        # Desvio próximo de 1
        stats['order_correlation'] > 0.7 and    # Preserva ordem relativa
        stats['outliers_ratio'] < 0.01         # Menos de 1% outliers
    )

    return is_valid, stats

print("✅ Normalizador melhorado implementado!")

✅ Normalizador melhorado implementado!


In [ ]:
# === BCI DATASET CLASS ===
class BCIDataset(Dataset):
    """Dataset PyTorch para dados EEG"""

    def __init__(self, windows: np.ndarray, labels: np.ndarray, transform=None, augment: bool = False):
        self.windows = torch.from_numpy(windows).float()
        self.labels = torch.from_numpy(labels).long()
        self.transform = transform
        self.augment = augment

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        window = self.windows[idx]
        label = self.labels[idx]

        if self.augment:
            # Adicionar ruído leve
            if torch.rand(1) < 0.3:
                noise = torch.randn_like(window) * 0.01
                window = window + noise

        if self.transform:
            window = self.transform(window)

        return window, label

print("✅ BCIDataset implementado!")

✅ BCIDataset implementado!


In [ ]:
# === EEG INCEPTION ERP MODEL ===
try:
    from braindecode.models import EEGInceptionERP
    print("✅ Braindecode EEGInceptionERP importado com sucesso!")
    BRAINDECODE_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erro ao importar braindecode: {e}")
    print("🔧 Vamos implementar um modelo CNN simples como fallback")
    BRAINDECODE_AVAILABLE = False

class EEGInceptionERPModel(nn.Module):
    """Wrapper para EEGInceptionERP com fallback para CNN simples"""

    def __init__(self, n_chans: int, n_outputs: int, n_times: int, sfreq: float = 125.0, **kwargs):
        super().__init__()
        self.n_chans = n_chans
        self.n_outputs = n_outputs
        self.n_times = n_times
        self.sfreq = sfreq
        self.is_trained = False

        if BRAINDECODE_AVAILABLE:
            try:
                self.model = EEGInceptionERP(
                    n_chans=n_chans,
                    n_outputs=n_outputs,
                    n_times=n_times,
                    sfreq=sfreq
                )
                self.model_type = "EEGInceptionERP"
                print(f"✅ Usando EEGInceptionERP: {n_chans} canais, {n_times} pontos temporais")
            except Exception as e:
                print(f"⚠️ Erro ao criar EEGInceptionERP: {e}")
                self.model = self._create_fallback_cnn(n_chans, n_outputs, n_times)
                self.model_type = "FallbackCNN"
        else:
            self.model = self._create_fallback_cnn(n_chans, n_outputs, n_times)
            self.model_type = "FallbackCNN"

    def _create_fallback_cnn(self, n_chans: int, n_outputs: int, n_times: int):
        """Criar CNN simples como fallback"""
        print(f"🔧 Criando CNN fallback: {n_chans} canais, {n_times} pontos temporais")
        return nn.Sequential(
            # Convolução temporal
            nn.Conv1d(n_chans, 32, kernel_size=25, padding=12),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.25),

            # Redução de dimensionalidade
            nn.Conv1d(32, 64, kernel_size=15, padding=7),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Dropout(0.25),

            # Convolução final
            nn.Conv1d(64, 128, kernel_size=7, padding=3),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),

            # Classificador
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, n_outputs)
        )

    def forward(self, x):
        if self.model_type == "EEGInceptionERP":
            return self.model(x)
        else:
            # Para o CNN fallback, precisamos transpor de (batch, channels, time) para (batch, channels, time)
            return self.model(x)

    def save(self, filepath: str):
        """Salvar modelo"""
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        save_dict = {
            'model_state_dict': self.model.state_dict(),
            'model_type': self.model_type,
            'n_chans': self.n_chans,
            'n_outputs': self.n_outputs,
            'n_times': self.n_times,
            'sfreq': self.sfreq,
            'is_trained': self.is_trained
        }
        torch.save(save_dict, filepath)
        print(f"✅ Modelo salvo: {filepath}")

    def load(self, filepath: str):
        """Carregar modelo"""
        checkpoint = torch.load(filepath, map_location='cpu')
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.is_trained = checkpoint.get('is_trained', True)
        print(f"✅ Modelo carregado: {filepath}")

print(f"✅ EEGInceptionERPModel implementado! Tipo: {'Braindecode' if BRAINDECODE_AVAILABLE else 'Fallback CNN'}")

✅ Braindecode EEGInceptionERP importado com sucesso!
✅ EEGInceptionERPModel implementado! Tipo: Braindecode


## 3. Configuração dos Caminhos e Parâmetros

In [ ]:
from pathlib import Path

# --- Raiz do seu trabalho no Drive ------------------------------------------------
PROJECT_ROOT = Path("/content/drive/MyDrive/Colab Notebooks/eeg_data")

# --- Dados ------------------------------------------------------------------------
EEG_DATA_PATH  = PROJECT_ROOT / "MNE-eegbci-data/files/eegmmidb/1.0.0"
DAVI_DATA_PATH = EEG_DATA_PATH / "Davi"

# --- Saída de modelos e resultados ------------------------------------------------
MODELS_PATH  = PROJECT_ROOT / "models"     # <-- ajuste se quiser outro local
RESULTS_PATH = PROJECT_ROOT / "results"

# Cria as pastas se ainda não existirem (evita erros de salvamento depois)
MODELS_PATH.mkdir(parents=True, exist_ok=True)
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

# --- Parâmetros de treinamento ---------------------------------------------------
TRAINING_PARAMS = {
    # Configurações de dados
    'exclude_subjects': ['Davi'],  # treina com todos exceto Davi
    'data_base_path': str(EEG_DATA_PATH),
    'model_save_path': str(MODELS_PATH),
    'results_save_path': str(RESULTS_PATH),

    # Configurações de treinamento
    'num_k_folds': 10,            # Número de folds para validação cruzada
    'num_epochs_per_fold': 30,    # Épocas máximas por fold
    'batch_size': 10,             # Tamanho do batch
    'early_stopping_patience': 8,  # Paciência para early stopping
    'learning_rate': 1e-3,        # Taxa de aprendizado
    'test_split_ratio': 0.2,      # Proporção do conjunto de teste

    # Identificação do modelo
    'model_name': 'eeg_inception_openbci_cv10',

    # Configurações de normalização
    'normalization': {
        'method': 'robust_zscore',
        'scope': 'channel',
        'outlier_threshold': 3.0
    }
}

print("✅ Configurações definidas:")
print(f"📂 Diretório de dados: {EEG_DATA_PATH}")
print(f"🏗️ Modelos: {MODELS_PATH}")
print(f"📊 Resultados: {RESULTS_PATH}")
print("\n📝 Parâmetros principais:")
for key, value in TRAINING_PARAMS.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for k, v in value.items():
            print(f"    - {k}: {v}")
    else:
        print(f"  - {key}: {value}")


✅ Configurações definidas:
📂 Diretório de dados: /content/drive/MyDrive/Colab Notebooks/eeg_data/MNE-eegbci-data/files/eegmmidb/1.0.0
🏗️ Modelos: /content/drive/MyDrive/Colab Notebooks/eeg_data/models
📊 Resultados: /content/drive/MyDrive/Colab Notebooks/eeg_data/results

📝 Parâmetros principais:
  - exclude_subjects: ['Davi']
  - data_base_path: /content/drive/MyDrive/Colab Notebooks/eeg_data/MNE-eegbci-data/files/eegmmidb/1.0.0
  - model_save_path: /content/drive/MyDrive/Colab Notebooks/eeg_data/models
  - results_save_path: /content/drive/MyDrive/Colab Notebooks/eeg_data/results
  - num_k_folds: 10
  - num_epochs_per_fold: 30
  - batch_size: 10
  - early_stopping_patience: 8
  - learning_rate: 0.001
  - test_split_ratio: 0.2
  - model_name: eeg_inception_openbci_cv10
  normalization:
    - method: robust_zscore
    - scope: channel
    - outlier_threshold: 3.0


## 4. Geração de Dados Sintéticos para Teste

Já que provavelmente você não tem os dados reais no Colab, vamos gerar dados sintéticos para testar o pipeline.

In [ ]:
# === CARREGAMENTO DE DADOS REAIS (OpenBCI CSV) ===
from pathlib import Path
import numpy as np
import pandas as pd
import mne
import re

def _read_openbci_csv(file_path: Path, sfreq: int = 125):
    """Carrega um único arquivo CSV gerado pela OpenBCI GUI e devolve um Raw MNE.
    Aprende automaticamente os nomes dos canais EXG… presentes no cabeçalho.
    Adiciona anotações (T0, T1, T2…) encontradas na coluna *Annotations*.
    """
    # Lê o CSV ignorando linhas iniciadas com '%'
    df = pd.read_csv(file_path, comment='%')

    # Seleciona colunas de EEG (por padrão 'EXG Channel 0‑15')
    eeg_cols = [c for c in df.columns if c.lower().startswith('exg channel')]
    data = df[eeg_cols].T.to_numpy() * 1e-6  # µV → V

    info = mne.create_info(ch_names=eeg_cols, sfreq=sfreq, ch_types='eeg')
    raw = mne.io.RawArray(data, info, verbose=False)

    # Anotações: mapeia strings 'T0','T1','T2'… para eventos
    if 'Annotations' in df.columns:
        onsets = df.index.to_numpy() / sfreq
        for onset, annot in zip(onsets, df['Annotations'].astype(str)):
            if annot.startswith('T'):
                raw.annotations.append(onset, 0, annot)
    return raw

def load_openbci_eeg(data_dir: Path):
    """Carrega janelas EEG (trials × canais × tempo) e rótulos binários
    (0: mão esquerda, 1: mão direita) a partir de arquivos CSV da OpenBCI.

    Arquitetura esperada:
        data_dir/
            S001R01_csv_openbci.csv
            S001R02_csv_openbci.csv
            ...
            Davi/
                Davi_R01_csv_openbci.csv
                ...
    O mapeamento de rótulos assume:
        'T1' → 0 (mão esquerda)
        'T2' → 1 (mão direita)
    Ajuste conforme a sua anotação real, se necessário.
    """
    windows, labels, subject_ids = [], [], []

    for csv_file in sorted(data_dir.rglob('*_csv_openbci.csv')):
        # Infer subject ID (primeiros 3 dígitos após 'S')
        match = re.search(r'[Ss](\d{3})', csv_file.stem)
        subj_id = int(match.group(1)) if match else 0

        raw = _read_openbci_csv(csv_file)

        # Extrai eventos T1/T2
        events, event_id = mne.events_from_annotations(raw,
                                                       event_id={'T1': 1, 'T2': 2},
                                                       verbose=False)
        if len(events) == 0:
            continue

        # Epoca intervalo 0‑3.2 s (≈ 400 amostras a 125 Hz)
        epochs = mne.Epochs(raw,
                            events,
                            event_id=event_id,
                            tmin=0,
                            tmax=3.2,
                            baseline=None,
                            preload=True,
                            verbose=False)

        lbl = epochs.events[:, 2] - 1  # Converte 1→0 (esq.), 2→1 (dir.)
        windows.append(epochs.get_data())
        labels.append(lbl)
        subject_ids.append(np.full(len(lbl), subj_id))

    if not windows:
        raise RuntimeError("Nenhuma janela EEG encontrada — verifique suas anotações T1/T2.")

    windows = np.concatenate(windows, axis=0)
    labels = np.concatenate(labels, axis=0)
    subject_ids = np.concatenate(subject_ids, axis=0)
    return windows, labels, subject_ids


print("📥 Carregando dataset OpenBCI CSV...")
windows, labels, subject_ids = load_openbci_eeg(EEG_DATA_PATH)
print(f"✅ Dados carregados: {windows.shape} janelas – {np.bincount(labels)} (classes) – {len(np.unique(subject_ids))} sujeitos")

📥 Carregando dataset OpenBCI CSV...
✅ Dados carregados: (3530, 16, 401) janelas – [1781 1749] (classes) – 79 sujeitos


## 5. Implementação do Pipeline de Treinamento Simplificado

In [ ]:
### Parâmetros de treinamento
TRAINING_PARAMS = {
    'exclude_subjects': ['Davi'],  # treina com todos exceto Davi
    'num_k_folds': 10,
    'num_epochs_per_fold': 30,
    'batch_size': 10,
    'early_stopping_patience': 8,
    'learning_rate': 1e-3,
    'test_split_ratio': 0.2,
    'model_name': 'eeg_inception_openbci_cv10',
}

# =============================================================================
# Dependências
# =============================================================================
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split, KFold

# =============================================================================
# Função principal de treinamento
# =============================================================================

def train_simplified_model(windows, labels, subject_ids, params):
    """Pipeline de treinamento simplificado com CV e early stopping."""

    print('🚀 Iniciando treinamento simplificado…')

    # --- Excluir sujeitos ------------------------------------------------------
    if isinstance(params.get('exclude_subjects'), list):
        mask = ~np.isin(subject_ids, params['exclude_subjects'])
        windows, labels, subject_ids = windows[mask], labels[mask], subject_ids[mask]

    print(f"📊 Amostras após exclusão: {windows.shape[0]}")

    # --- Normalização ----------------------------------------------------------
    print("\n📊 Configurando normalização...")
    normalizer = ImprovedEEGNormalizer(
        method='robust_zscore',  # mais robusto a outliers
        scope='channel',         # normaliza por canal
        outlier_threshold=3.0    # 3 desvios padrão para outliers
    )

    # Normalizar dados
    windows_norm = normalizer.fit_transform(windows)

    # Validar normalização
    is_valid, norm_stats = validate_normalization(windows, windows_norm)
    print("\n🔍 Validação da Normalização:")
    print(f"  - Média: {norm_stats['mean']:.3f} (ideal: próximo de 0)")
    print(f"  - Desvio: {norm_stats['std']:.3f} (ideal: próximo de 1)")
    print(f"  - Correlação de ordem: {norm_stats['order_correlation']:.3f}")
    print(f"  - Proporção de outliers: {norm_stats['outliers_ratio']*100:.2f}%")
    print(f"  - Status: {'✅ OK' if is_valid else '⚠️ Verificar'}")

    # --- Split treino/val/test -------------------------------------------------
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        windows_norm, labels,
        test_size=params['test_split_ratio'],
        random_state=42,
        stratify=labels
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n🖥️ Device: {device}")

    # --- Validação cruzada -----------------------------------------------------
    kfold = KFold(n_splits=params['num_k_folds'], shuffle=True, random_state=42)
    fold_accs = []
    best_model = None
    best_overall_acc = 0.0

    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_val), 1):
        print(f"\n📁 Fold {fold}/{params['num_k_folds']}")

        X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
        y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

        # Usar augmentation apenas no treino
        train_ds = BCIDataset(X_train, y_train, augment=True)
        val_ds   = BCIDataset(X_val,   y_val,   augment=False)

        train_loader = DataLoader(train_ds, batch_size=params['batch_size'], shuffle=True)
        val_loader   = DataLoader(val_ds,   batch_size=params['batch_size'], shuffle=False)

        model = EEGInceptionERPModel(
            n_chans=windows.shape[1],
            n_outputs=len(np.unique(labels)),
            n_times=windows.shape[2]
        ).to(device)

        criterion = nn.CrossEntropyLoss()
        optim     = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

        best_val_acc, patience = 0.0, 0
        fold_history = []

        for epoch in range(params['num_epochs_per_fold']):
            # ---- Treinamento ----
            model.train()
            train_loss = 0.0
            correct = total = 0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optim.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                optim.step()

                train_loss += loss.item()
                pred = out.argmax(1)
                correct += (pred == yb).sum().item()
                total   += yb.size(0)

            train_acc = correct / total
            train_loss = train_loss / len(train_loader)

            # ---- Validação ----
            model.eval()
            val_loss = 0.0
            correct = total = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    out = model(xb)
                    val_loss += criterion(out, yb).item()
                    pred = out.argmax(1)
                    correct += (pred == yb).sum().item()
                    total   += yb.size(0)

            val_acc = correct / total
            val_loss = val_loss / len(val_loader)

            fold_history.append({
                'epoch': epoch + 1,
                'train_acc': train_acc,
                'train_loss': train_loss,
                'val_acc': val_acc,
                'val_loss': val_loss
            })

            print(f"  Epoch {epoch+1}: Train {train_acc:.3f} ({train_loss:.3f}) | Val {val_acc:.3f} ({val_loss:.3f})")

            # ---- Early stopping ----
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience = 0
                # Guardar melhor modelo geral
                if val_acc > best_overall_acc:
                    best_overall_acc = val_acc
                    best_model = model.state_dict()
            else:
                patience += 1
                if patience >= params['early_stopping_patience']:
                    print(f"  ⏹️  Early stopping na época {epoch+1}")
                    break

        fold_accs.append(best_val_acc)
        print(f"✅ Fold {fold} concluído – Melhor Val Acc: {best_val_acc:.4f}")

    # --- Avaliação no conjunto de teste ----------------------------------------
    print("\n🎯 Avaliando no conjunto de teste...")
    if best_model is not None:
        model.load_state_dict(best_model)
        test_ds = BCIDataset(X_test, y_test, augment=False)
        test_loader = DataLoader(test_ds, batch_size=params['batch_size'], shuffle=False)

        model.eval()
        test_loss = 0.0
        correct = total = 0
        predictions = []
        true_labels = []

        with torch.no_grad():
            for xb, yb in test_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                test_loss += criterion(out, yb).item()
                pred = out.argmax(1)
                correct += (pred == yb).sum().item()
                total += yb.size(0)

                predictions.extend(pred.cpu().numpy())
                true_labels.extend(yb.cpu().numpy())

        test_acc = correct / total
        test_loss = test_loss / len(test_loader)
    else:
        test_acc = test_loss = 0.0
        predictions = []
        true_labels = []

    # Preparar resultados estruturados
    cv_mean = np.mean(fold_accs)
    cv_std = np.std(fold_accs)
    print(f"\n📊 Resultados Finais:")
    print(f"  CV Mean Acc: {cv_mean:.4f} ± {cv_std:.4f}")
    print(f"  Test Acc: {test_acc:.4f} (loss: {test_loss:.4f})")

    results = {
        'fold_accuracies': fold_accs,
        'cv_mean_accuracy': cv_mean,
        'cv_std_accuracy': cv_std,
        'final_test_accuracy': test_acc,
        'test_loss': test_loss,
        'model_name': params['model_name'],
        'best_model_state': best_model,
        'normalization_stats': normalizer.get_stats(),
        'normalization_validation': norm_stats,
        'training_params': params,
        'predictions': predictions,
        'true_labels': true_labels
    }

    return results


## 6. Execução do Treinamento Principal

In [ ]:
# === EXECUÇÃO DO TREINAMENTO ===
print("🚀 INICIANDO TREINAMENTO PRINCIPAL...")

try:
    # Executar treinamento com dados sintéticos
    main_results = train_simplified_model(
        windows,
        labels,
        subject_ids,
        MAIN_TRAINING_PARAMS
    )

    print("\n📊 RESULTADOS DO TREINAMENTO:")
    print(f"  - CV Mean: {main_results['cv_mean_accuracy']:.4f} ± {main_results['cv_std_accuracy']:.4f}")
    print(f"  - Test Acc: {main_results['final_test_accuracy']:.4f}")
    print(f"  - Modelo: {main_results['model_name']}")

    # Salvar resultados
    results_path = RESULTS_PATH / "main_training_results.json"
    RESULTS_PATH.mkdir(exist_ok=True)
    with open(results_path, 'w') as f:
        json.dump(main_results, f, indent=2, default=str)

    print(f"💾 Resultados salvos em: {results_path}")

except Exception as e:
    print(f"❌ Erro durante treinamento: {e}")
    import traceback
    traceback.print_exc()
    main_results = None

🚀 INICIANDO TREINAMENTO PRINCIPAL...
❌ Erro durante treinamento: name 'MAIN_TRAINING_PARAMS' is not defined


Traceback (most recent call last):
  File "<ipython-input-12-1593755867>", line 10, in <cell line: 0>
    MAIN_TRAINING_PARAMS
NameError: name 'MAIN_TRAINING_PARAMS' is not defined


## 7. Visualização dos Resultados

In [ ]:
# === VISUALIZAÇÃO DOS RESULTADOS ===
if main_results:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Resultados do Treinamento BCI - Google Colab', fontsize=16, fontweight='bold')

    # 1. Acurácias por fold
    ax1 = axes[0, 0]
    fold_accs = main_results['fold_accuracies']
    ax1.bar(range(1, len(fold_accs) + 1), fold_accs, alpha=0.7, color='skyblue')
    ax1.axhline(y=main_results['cv_mean_accuracy'], color='red', linestyle='--',
                label=f"Mean: {main_results['cv_mean_accuracy']:.3f}")
    ax1.set_xlabel('Fold')
    ax1.set_ylabel('Acurácia')
    ax1.set_title('Acurácia por Fold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Comparação CV vs Test
    ax2 = axes[0, 1]
    models = ['CV Mean', 'Test Final']
    accuracies = [main_results['cv_mean_accuracy'], main_results['final_test_accuracy']]
    bars = ax2.bar(models, accuracies, color=['lightcoral', 'lightgreen'], alpha=0.8)
    ax2.set_ylabel('Acurácia')
    ax2.set_title('CV vs Test Performance')
    ax2.set_ylim(0, 1)

    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

    # 3. Distribuição das classes sintéticas
    ax3 = axes[1, 0]
    unique, counts = np.unique(labels, return_counts=True)
    ax3.pie(counts, labels=[f'Classe {u}' for u in unique], autopct='%1.1f%%')
    ax3.set_title('Distribuição de Classes')

    # 4. Resumo textual
    ax4 = axes[1, 1]
    ax4.axis('off')
    summary_text = f"""
    RESUMO DO TREINAMENTO

    Dataset: Sintético
    Subjects: {len(np.unique(subject_ids))}
    Amostras: {len(labels)}

    Modelo: {main_results['model_name']}
    Tipo: {'EEGInceptionERP' if BRAINDECODE_AVAILABLE else 'CNN Fallback'}

    Performance:
    • CV Mean: {main_results['cv_mean_accuracy']:.4f}
    • CV Std: {main_results['cv_std_accuracy']:.4f}
    • Test Acc: {main_results['final_test_accuracy']:.4f}

    Status: ✅ Sucesso
    """
    ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes,
             fontsize=10, verticalalignment='top', fontfamily='monospace')

    plt.tight_layout()
    plt.savefig(RESULTS_PATH / 'colab_training_results.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("✅ Visualização criada e salva!")
else:
    print("❌ Não há resultados para visualizar")

❌ Não há resultados para visualizar


# 9. Fine-tuning com Dados Específicos

Nesta seção, vamos realizar o fine-tuning do modelo treinado anteriormente usando os dados específicos localizados na pasta Davi.

O processo de fine-tuning envolve:
1. Carregar o melhor modelo do treinamento anterior
2. Congelar parte das camadas iniciais (transfer learning)
3. Treinar apenas as últimas camadas com os novos dados
4. Validar a performance no conjunto de teste específico

Este processo é especialmente útil quando temos um modelo base treinado em muitos sujeitos e queremos adaptá-lo para um sujeito específico.

In [ ]:
# === FINE-TUNING COM DADOS ESPECÍFICOS ===
def fine_tune_model(base_model, windows, labels, params, original_norm_stats=None):
    """Realiza fine-tuning do modelo base com dados específicos."""
    print("🎯 Iniciando fine-tuning...")

    # --- Preparação dos dados ------------------------------------------------
    if original_norm_stats and params.get('use_original_normalization', False):
        print("📊 Usando normalização do treino original...")
        # Verificar quais estatísticas temos disponíveis
        if 'median' in original_norm_stats and 'iqr' in original_norm_stats:
            windows_norm = (windows - original_norm_stats['median']) / original_norm_stats['iqr']
        elif 'mean' in original_norm_stats and 'std' in original_norm_stats:
            windows_norm = (windows - original_norm_stats['mean']) / original_norm_stats['std']
        else:
            print("⚠️ Estatísticas de normalização original não reconhecidas, usando nova normalização...")
            normalizer = ImprovedEEGNormalizer(method='robust_zscore')
            windows_norm = normalizer.fit_transform(windows)
    else:
        print("📊 Usando nova normalização específica...")
        normalizer = ImprovedEEGNormalizer(method='robust_zscore')
        windows_norm = normalizer.fit_transform(windows)

        # Comparar estatísticas se tivermos as originais
        if original_norm_stats:
            # Pegar estatísticas atuais
            current_stats = normalizer.get_stats()

            # Comparar estatísticas disponíveis
            if 'median' in current_stats and 'median' in original_norm_stats:
                median_diff = np.abs(current_stats['median'] - original_norm_stats['median']).mean()
                iqr_diff = np.abs(current_stats['iqr'] - original_norm_stats['iqr']).mean()
                print(f"📈 Diferença média nas estatísticas (robust):")
                print(f"   - Mediana: {median_diff:.6f}")
                print(f"   - IQR: {iqr_diff:.6f}")
            elif 'mean' in current_stats and 'mean' in original_norm_stats:
                mean_diff = np.abs(current_stats['mean'] - original_norm_stats['mean']).mean()
                std_diff = np.abs(current_stats['std'] - original_norm_stats['std']).mean()
                print(f"📈 Diferença média nas estatísticas (zscore):")
                print(f"   - Média: {mean_diff:.6f}")
                print(f"   - Desvio: {std_diff:.6f}")

    # Validar normalização
    is_valid, norm_stats = validate_normalization(windows, windows_norm)
    print("\n🔍 Validação da Normalização:")
    print(f"  - Média: {norm_stats['mean']:.3f} (ideal: próximo de 0)")
    print(f"  - Desvio: {norm_stats['std']:.3f} (ideal: próximo de 1)")
    print(f"  - Correlação de ordem: {norm_stats['order_correlation']:.3f}")
    print(f"  - Proporção de outliers: {norm_stats['outliers_ratio']*100:.2f}%")
    print(f"  - Status: {'✅ OK' if is_valid else '⚠️ Verificar'}")

    # --- Split train/val/test -------------------------------------------------
    X_train, X_temp, y_train, y_temp = train_test_split(
        windows_norm, labels,
        test_size=0.3,
        random_state=42,
        stratify=labels
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=0.5,
        random_state=42,
        stratify=y_temp
    )

    # Datasets
    train_ds = BCIDataset(X_train, y_train, augment=True)
    val_ds = BCIDataset(X_val, y_val, augment=False)
    test_ds = BCIDataset(X_test, y_test, augment=False)

    # Dataloaders
    train_loader = DataLoader(train_ds, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=params['batch_size'], shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=params['batch_size'], shuffle=False)

    # --- Preparação do modelo -----------------------------------------------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = base_model.to(device)

    # Congelar camadas iniciais conforme estratégia
    if params['freeze_strategy'] == 'early':
        frozen_count = 0
        for name, param in model.named_parameters():
            if 'encoder' in name or 'conv' in name:
                param.requires_grad = False
                frozen_count += 1
        print(f"\n🧊 Camadas congeladas: {frozen_count}")

    # Otimizador com learning rate reduzido
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(
        trainable_params,
        lr=params['learning_rate_ratio'] * params['learning_rate']
    )
    criterion = nn.CrossEntropyLoss()

    # --- Fine-tuning -------------------------------------------------------
    best_val_acc = 0.0
    best_model_state = None
    patience = params.get('early_stopping_patience', 5)
    wait = 0

    history = []

    for epoch in range(params['epochs']):
        # Treino
        model.train()
        train_loss = train_correct = train_total = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            pred = out.argmax(1)
            train_correct += (pred == yb).sum().item()
            train_total += yb.size(0)

        train_acc = train_correct / train_total
        train_loss = train_loss / len(train_loader)

        # Validação
        model.eval()
        val_loss = val_correct = val_total = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                pred = out.argmax(1)

                val_loss += loss.item()
                val_correct += (pred == yb).sum().item()
                val_total += yb.size(0)

        val_acc = val_correct / val_total
        val_loss = val_loss / len(val_loader)

        # Guardar histórico
        history.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        })

        print(f"Época {epoch+1}: Train {train_acc:.3f} ({train_loss:.3f}) | Val {val_acc:.3f} ({val_loss:.3f})")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"\n⏹️ Early stopping na época {epoch+1}")
                break

    # --- Avaliação final --------------------------------------------------
    print("\n🎯 Avaliando modelo final...")
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    model.eval()
    test_loss = test_correct = test_total = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            pred = out.argmax(1)

            test_loss += loss.item()
            test_correct += (pred == yb).sum().item()
            test_total += yb.size(0)

            predictions.extend(pred.cpu().numpy())
            true_labels.extend(yb.cpu().numpy())

    test_acc = test_correct / test_total
    test_loss = test_loss / len(test_loader)

    results = {
        'final_test_accuracy': test_acc,
        'best_val_accuracy': best_val_acc,
        'test_loss': test_loss,
        'model_state': best_model_state,
        'history': history,
        'predictions': predictions,
        'true_labels': true_labels,
        'normalization_validation': norm_stats
    }

    print(f"\n✨ Resultados do Fine-tuning:")
    print(f"   - Melhor Val Acc: {best_val_acc:.4f}")
    print(f"   - Test Acc: {test_acc:.4f}")
    print(f"   - Test Loss: {test_loss:.4f}")

    return results

# === EXECUÇÃO DO FINE-TUNING ===
print("🔄 Carregando dados do Davi...")

# Carregar dados específicos do Davi
davi_windows, davi_labels, _ = load_openbci_eeg(Path(DAVI_DATA_PATH))
print(f"📊 Dados carregados: {davi_windows.shape[0]} amostras")

# Parâmetros do fine-tuning
fine_tuning_params = {
    'freeze_strategy': 'early',      # congela camadas iniciais
    'learning_rate_ratio': 0.1,      # lr menor para fine-tuning
    'epochs': 30,
    'batch_size': 8,
    'early_stopping_patience': 5,
    'learning_rate': TRAINING_PARAMS['learning_rate'],
    'use_original_normalization': False  # Escolha se quer usar normalização original ou nova
}

# Executar fine-tuning
if main_results and main_results.get('best_model_state') is not None:
    print("\n🎯 Iniciando fine-tuning com modelo base...")

    # Criar modelo base com os pesos do melhor modelo
    base_model = EEGInceptionERPModel(
        n_chans=davi_windows.shape[1],
        n_outputs=len(np.unique(davi_labels)),
        n_times=davi_windows.shape[2]
    )
    base_model.load_state_dict(main_results['best_model_state'])

    # Fine-tuning
    ft_results = fine_tune_model(
        base_model=base_model,
        windows=davi_windows,
        labels=davi_labels,
        params=fine_tuning_params,
        original_norm_stats=main_results.get('normalization_stats')
    )

    # Salvar modelo final
    if ft_results['model_state'] is not None:
        base_model.load_state_dict(ft_results['model_state'])
        model_path = MODELS_PATH / f"{TRAINING_PARAMS['model_name']}_finetuned.pt"
        torch.save({
            'model_state_dict': ft_results['model_state'],
            'test_accuracy': ft_results['final_test_accuracy'],
            'val_accuracy': ft_results['best_val_accuracy'],
            'test_loss': ft_results['test_loss'],
            'history': ft_results['history'],
            'normalization_validation': ft_results['normalization_validation']
        }, model_path)
        print(f"\n💾 Modelo fine-tuned salvo em: {model_path}")
else:
    print("❌ Modelo base não disponível para fine-tuning")

🔄 Carregando dados do Davi...
📊 Dados carregados: 20 amostras
❌ Modelo base não disponível para fine-tuning


## 8. Relatório Final

Resumo da execução no Google Colab com implementação inline.

In [ ]:
print("=" * 60)
print("            RELATÓRIO FINAL - GOOGLE COLAB")
print("=" * 60)

print(f"\n🖥️ AMBIENTE:")
print(f"  - Plataforma: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"  - Braindecode: {'✅ Disponível' if BRAINDECODE_AVAILABLE else '❌ Fallback CNN'}")
print(f"  - Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

print(f"\n📊 DADOS:")
print(f"  - Tipo: Sintético")
print(f"  - Amostras: {len(labels)}")
print(f"  - Subjects: {len(np.unique(subject_ids))}")
print(f"  - Classes: {len(np.unique(labels))}")

if main_results:
    print(f"\n🎯 RESULTADOS:")
    print(f"  - Modelo: {main_results['model_name']}")
    print(f"  - CV Accuracy: {main_results['cv_mean_accuracy']:.4f} ± {main_results['cv_std_accuracy']:.4f}")
    print(f"  - Test Accuracy: {main_results['final_test_accuracy']:.4f}")
    print(f"  - Status: ✅ SUCESSO")
else:
    print(f"\n❌ RESULTADOS: FALHOU")

print(f"\n📁 ARQUIVOS CRIADOS:")
for path in [MODELS_PATH, RESULTS_PATH]:
    if path.exists():
        files = list(path.rglob("*"))
        print(f"  - {path.name}/: {len(files)} arquivo(s)")

print(f"\n🎉 CONCLUSÃO:")
print(f"  Este notebook demonstrou como adaptar o pipeline BCI")
print(f"  para execução no Google Colab, incluindo:")
print(f"  • Implementação inline das classes principais")
print(f"  • Geração de dados sintéticos para teste")
print(f"  • Pipeline de treinamento simplificado")
print(f"  • Fallback para quando braindecode não está disponível")

print(f"\n✨ Execução concluída em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

            RELATÓRIO FINAL - GOOGLE COLAB

🖥️ AMBIENTE:
  - Plataforma: Google Colab
  - Braindecode: ✅ Disponível
  - Device: cpu

📊 DADOS:
  - Tipo: Sintético
  - Amostras: 3530
  - Subjects: 79
  - Classes: 2

❌ RESULTADOS: FALHOU

📁 ARQUIVOS CRIADOS:
  - models/: 6 arquivo(s)
  - results/: 2 arquivo(s)

🎉 CONCLUSÃO:
  Este notebook demonstrou como adaptar o pipeline BCI
  para execução no Google Colab, incluindo:
  • Implementação inline das classes principais
  • Geração de dados sintéticos para teste
  • Pipeline de treinamento simplificado
  • Fallback para quando braindecode não está disponível

✨ Execução concluída em: 2025-06-16 03:23:59
